# Problem Statement: **Building a Smart Florist Assistant for AtliQ's Online Plant Store**

### AtliQ is launching a new online plant and flower store aimed at making it easier for customers to select and purchase flowers. To enhance the shopping experience, AtliQ wants to implement an AI-based flower classification assistant that can identify flower types from uploaded images and provide details about them, such as availability, pricing, and care instructions. Your goal is to create and train a classification model that can accurately classify images of flowers into 102 categories using the **Oxford Flowers102 dataset**.

**Pre-trained Model**: Use a model pre-trained on ImageNet (like ResNet18).

**Transfer Learning Steps**:
* Load the pre-trained ResNet18.
* Freeze the early layers (feature extractor).
* Replace the final classification layer with a new one for flower classification.
* Fine-tune the model on the target dataset (Oxford Flowers).

Imports and CUDA

In [1]:
# Import the required libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


**Step1:** Dataset Overview

* The Oxford 102 Flowers dataset consists of 102 flower categories. Let's load the dataset and apply necessary transformations such as resizing, cropping, and normalizing.
* use **batch_size=32** in dataloaders

In [12]:
# Define transformations for the train and validation sets
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the images to match ResNet input
    transforms.ToTensor(),          # Convert images to PyTorch tensors
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize with ImageNet stats
])

# Download and load the training and validation datasets
train_dataset = datasets.Flowers102(root='data', split='train', download=True, transform=transform)
val_dataset = datasets.Flowers102(root='data', split='val', download=True, transform=transform)

# Create DataLoader for batch processing
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


**Key Insights**:

* We resize the images to 224*224 to match ResNet's input size, convert them to tensors, and normalize them using ImageNet's mean and standard deviation.



---



### **Step2**: Model Training without Transfer Learning (with CNNs)

* Build a CNN from scratch to classify images in the Flowers 102 dataset into 102 categories using the PyTorch nn.Sequential module.

**Architecture:**

Two convolutional blocks:
* Block 1: 32 filters, 3x3 kernel, ReLU, 2x2 MaxPool.
* Block 2: 64 filters, 3x3 kernel, ReLU, 2x2 MaxPool.
* After convolutions, the feature map size will be 64 x 56 x 56.
*` nn.Flatten()` is used to flatten the feature maps into a vector for input to the classifier block.

Classifier Block:
* Fully connected layer with 512 neurons.
* Dropout added with a probability of 0.5 to prevent overfitting.
* Final fully connected layer with 102 neurons (for 102 categories) and `nn.LogSoftmax.`

Use `nn.CrossEntropyLoss` as the loss function.

Use `optim.Adam` as the optimizer with a learning rate of 0.0001.

In [13]:
class FlowerCNN(nn.Module):
    def __init__(self, num_classes=102):
        super().__init__()
        self.features = nn.Sequential(
            # First conv block
            nn.Conv2d(3, 32, kernel_size=(3,3), padding="same"),
            nn.ReLU(),
            nn.MaxPool2d(stride=(2,2), kernel_size=(2,2)),

            # Second conv block
            nn.Conv2d(32, 64, kernel_size=(3,3), padding="same"),
            nn.ReLU(),
            nn.MaxPool2d(stride=(2,2), kernel_size=(2,2)),

            # Flatten Layer
            nn.Flatten()
        )

        # Classifier with LogSoftmax
        self.classifier = nn.Sequential(
            nn.Linear(64*56*56, 512),
            nn.ReLU(),
            nn.Dropout(0.5),  # Add dropout
            nn.Linear(512, num_classes),
            nn.LogSoftmax(dim=1)
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

# Initialize model, criterion, and optimizer
model_scratch = FlowerCNN().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_scratch.parameters(), lr=0.0001)



---



**Step3**: Model Training.

* num_epochs = 8

In [14]:
# Training function
def train_model_scratch(model_scratch, train_loader, loss_fn, optimizer, num_epochs=8):
    model_scratch.train()

    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model_scratch(inputs)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100 * correct / total
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%')

# Train the model from scratch
train_model_scratch(model_scratch, train_loader, loss_fn, optimizer, num_epochs=8)


Epoch [1/8], Loss: 4.9140, Accuracy: 1.57%
Epoch [2/8], Loss: 4.4063, Accuracy: 4.90%
Epoch [3/8], Loss: 4.0750, Accuracy: 8.24%
Epoch [4/8], Loss: 3.6127, Accuracy: 16.47%
Epoch [5/8], Loss: 3.0374, Accuracy: 29.02%
Epoch [6/8], Loss: 2.5471, Accuracy: 37.55%
Epoch [7/8], Loss: 1.9753, Accuracy: 52.25%
Epoch [8/8], Loss: 1.5748, Accuracy: 63.43%


**Key Insights:**

* model.train() sets the model to training mode.

For each batch, we:
* Move the inputs and labels to the device (GPU or CPU).
* Perform a forward pass to get the predictions.
* Compute the loss using criterion.
* Perform backpropagation with loss.backward().
* Update the model's parameters with optimizer.step().
* After each epoch, we calculate and print the loss and accuracy



---



**Step4**: Model Evauation

In [15]:
def evaluate_model(model_scratch, val_loader):
    # Evaluation loop
    model_scratch.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs =model_scratch(inputs)
            _, predicted = torch.max(outputs,1)
            total += labels.size(0)
            correct += (predicted==labels).sum.itme()

    print(f"Test Accuracy: {100 * correct / total:.2f}%")

evaluate_model(model_scratch, val_loader)

AttributeError: 'builtin_function_or_method' object has no attribute 'itme'



---



### **Step5**: Model Training with Transfer Learning!

**Loading the Pre-Trained ResNet18 Model**

We will load a pre-trained ResNet18 model and freeze all the layers except the final classification layer. This means the model will use previously learned features and only adapt the final layer for our flower classification task.

In [16]:
# Load pre-trained ResNet18 model with proper weights specification
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

# Freeze all the convolutional layers (feature extractor)
for param in model.parameters():
    param.requires_grad = False

# Replace the final fully connected layer (for 1000 ImageNet classes) with one for 102 flowers
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 102)

# Move the model to the GPU if available
model = model.to(device)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\nande/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:01<00:00, 24.8MB/s]


**Key Insights:**

* We load the pre-trained ResNet18 model.
* Freeze the layers by setting param.requires_grad = False. This prevents the early layers from being updated during backpropagation.
* Replace the final fully connected layer (model.fc) to have 102 output classes instead of 1000 (for ImageNet).



---



**Step6:** Setting Up the Loss Function and Optimizer

Now that we have the model ready, let's set up the loss function (CrossEntropyLoss) and optimizer (Adam, lr=0.001). We'll only optimize the new fully connected layer.


In [17]:
# Define loss function and optimizer (only for the last layer)
loss_fn = nn.CrossEntropyLoss()

# Since we're only training the final layer, only pass its parameters to the optimizer
optimizer = optim.Adam(model.parameters(),lr=0.001)




---



**Step8:** Training the Model

Now, let's define the training loop. We will iterate over the dataset, calculate the loss, perform backpropagation, and update the model weights for the final layer.

* `num_epochs` = 6

In [18]:
# Training function
def train_model(model, train_loader, loss_fn, optimizer, num_epochs=6):
    model.train()  # Set the model to training mode

    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        # Iterate over batches
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = loss_fn(outputs,labels)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            # Track training loss
            running_loss += loss
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted==labels).sum().item()

        # Print statistics after each epoch
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100 * correct/total
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%')

# Train the model for 6 epochs
train_model(model, train_loader, loss_fn, optimizer, num_epochs=6)


Epoch [1/6], Loss: 4.5988, Accuracy: 4.51%
Epoch [2/6], Loss: 3.2298, Accuracy: 43.82%
Epoch [3/6], Loss: 2.3039, Accuracy: 73.53%
Epoch [4/6], Loss: 1.6236, Accuracy: 87.25%
Epoch [5/6], Loss: 1.1573, Accuracy: 92.06%
Epoch [6/6], Loss: 0.8680, Accuracy: 95.59%




---



**Step9:** Evaluating the Model

After training, we evaluate the model on the validation set to check how well it has learned to classify flowers.

In [20]:
# Evaluation function
def evaluate_model(model, val_loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # No need to compute gradients for validation
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, predicted = (torch.max(outputs,1))
            total += labels.size(0)
            correct +=(predicted==labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Validation Accuracy: {accuracy:.2f}%')

# Evaluate the model
evaluate_model(model, val_loader)

Validation Accuracy: 78.14%




---



**End Result:**

By using transfer learning, we leveraged a ResNet18 model pre-trained on ImageNet to classify flowers with minimal training time and data.